In [20]:
import requests
import boto3
import mlflow
from mlflow import pyfunc as ml_pyfunc
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn import metrics

## Import dataset
leads_dataset = pd.read_csv('synthetic_leads.csv')
leads_dataset.columns = map(str.lower, leads_dataset.columns)

In [21]:
# Create data pre-processing steps before plugging into model
leads_categorical_columns = ['lead_source',
                             'country',
                             'gender',
                             'education_level',
                             'occupation',
                             'industry',
                             'initial_response',
                             'general_knowledge',
                             'business_knowledge',
                             'company_size',
                             'lead_quality']

leads_numeric_columns = ['age',
                         'income',
                         'total_calls_attended',
                         'total_meetings_attended',
                         'company_estimated_revenue',]

leads_response_columns = ['lead_score']

In [22]:
#split data for training, remove extras

leads_x = leads_dataset.drop(leads_response_columns, axis=1)
leads_y = leads_dataset[leads_response_columns]

leads_x_train, leads_x_test, leads_y_train, leads_y_test = train_test_split(leads_x,
                                                                            leads_y,
                                                                            train_size=0.7,
                                                                            test_size=0.3,
                                                                            random_state=5050)
leads_x_train.head()

,lead_source,country,age,gender,education_level,occupation,industry,income,initial_response,do_not_contact,total_calls_attended,total_meetings_attended,general_knowledge,business_knowledge,company_size,company_estimated_revenue,lead_quality
7072,OTHER,Canada,39,OTHER,PhD,EMPLOYEE,SERVICES,99000,POSITIVE,No,4,2,ADVANCED,ADVANCED,MEDIUM,2654000,HOT
4141,PAID,USA,51,OTHER,BACHELOR,BUSINESSMAN,RETAIL,153000,POSITIVE,No,3,3,ADVANCED,INTERMEDIATE,MEDIUM,2101000,HOT
6227,OTHER,Australia,38,FEMALE,COLLEGE,EMPLOYEE,RETAIL,93000,NEUTRAL,No,3,2,ADVANCED,INTERMEDIATE,MEDIUM,1605000,WARM
9804,OTHER,France,37,OTHER,BACHELOR,UNEMPLOYED,FINANCE,73000,NEGATIVE,Yes,1,0,EXPERT,BASIC,SMALL,744000,COLD
7930,OTHER,Pakistan,28,OTHER,MASTER,OTHER,FINANCE,87000,NEGATIVE,Yes,2,0,EXPERT,NOVICE,SMALL,619000,COLD


In [23]:
scaler = StandardScaler()
scaler = scaler.fit(leads_x_train[leads_numeric_columns])

In [24]:
def pre_process_leads_data(df,
                           numeric_columns,
                           categorical_columns,
                           fitted_scaler,
                           train_df_columns = None):
    ## create new df with selected columns
    df.columns = map(str.lower, df.columns)
    _df = df[list(numeric_columns + categorical_columns)].copy()

    ## scale the numeric columns with the pre-built scaler
    _df[numeric_columns] = fitted_scaler.transform(_df[numeric_columns])

    # First, make categorical text lowercase
    _df[categorical_columns] = _df[categorical_columns].apply(lambda x: x.str.lower())
    # Next, create one-hot-encoded variables, add to dataframe, drop old columns
    _df_dummies = pd.get_dummies(_df[categorical_columns], drop_first=True)
    _df = pd.concat([_df, _df_dummies], axis=1)
    _df.drop(categorical_columns, axis=1, inplace = True)

    if train_df_columns:
        _df = _df.reindex(columns=train_df_columns, fill_value=0)

    return _df

In [25]:
leads_x_train_clean = pre_process_leads_data(df = leads_x_train,
                                            numeric_columns = leads_numeric_columns,
                                            categorical_columns = leads_categorical_columns,
                                            fitted_scaler = scaler)

leads_x_test_clean = pre_process_leads_data(df = leads_x_test,
                                           numeric_columns = leads_numeric_columns,
                                           categorical_columns = leads_categorical_columns,
                                           fitted_scaler = scaler,
                                           train_df_columns = leads_x_train_clean.columns.tolist())

In [28]:
leads_x_train_clean.head()

,age,income,total_calls_attended,total_meetings_attended,company_estimated_revenue,lead_source_organic,lead_source_other,lead_source_paid,lead_source_referral,country_brazil,...,general_knowledge_intermediate,general_knowledge_novice,business_knowledge_basic,business_knowledge_expert,business_knowledge_intermediate,business_knowledge_novice,company_size_medium,company_size_small,lead_quality_hot,lead_quality_warm
7072,-0.143605,-0.280340,0.135095,0.227912,-0.515709,False,True,False,False,False,...,False,False,False,False,False,False,True,False,True,False
4141,1.093861,0.902358,-0.423809,1.145328,-0.693591,False,False,True,False,False,...,False,False,False,False,True,False,True,False,True,False
6227,-0.246727,-0.411751,-0.423809,0.227912,-0.853138,False,True,False,False,False,...,False,False,False,False,True,False,True,False,False,True
9804,-0.349849,-0.849787,-1.541618,-1.606919,-1.130094,False,True,False,False,False,...,False,False,True,False,False,False,False,True,False,False
7930,-1.277948,-0.543162,-0.982714,-1.606919,-1.170302,False,True,False,False,False,...,False,False,False,False,False,True,False,True,False,False


In [31]:
logm1 = sm.GLM(leads_y_train.astype(float),(sm.add_constant(leads_x_test_clean)), family = sm.families.Binomial())
logm1.fit().summary()

ValueError: Pandas data cast to numpy dtype of object. Check apiInput data with np.asarray(data). The types seen werelead_score    float64
dtype: object and const                              float64
age                                float64
income                             float64
total_calls_attended               float64
total_meetings_attended            float64
company_estimated_revenue          float64
lead_source_organic                   bool
lead_source_other                     bool
lead_source_paid                      bool
lead_source_referral                  bool
country_brazil                        bool
country_canada                        bool
country_china                         bool
country_france                        bool
country_germany                       bool
country_india                         bool
country_japan                         bool
country_pakistan                      bool
country_uk                            bool
country_usa               

In [32]:
logreg = LogisticRegression()

In [33]:
rfe = RFE(estimator=logreg, n_features_to_select= 15)
rfe.fit(leads_x_train_clean,leads_y_train)

C:\Users\hp\PycharmProjects\leadScoringModel\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp\PycharmProjects\leadScoringModel\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\hp\PycharmProjects\leadScoringModel\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array 

RFE(estimator=LogisticRegression(), n_features_to_select=15)

In [34]:
rfe_col = list(leads_x_train_clean.columns[rfe.support_])
rfe_col

['age',
 'income',
 'total_calls_attended',
 'total_meetings_attended',
 'company_estimated_revenue',
 'lead_source_other',
 'lead_source_referral',
 'education_level_master',
 'occupation_employee',
 'occupation_unemployed',
 'initial_response_neutral',
 'initial_response_positive',
 'company_size_medium',
 'lead_quality_hot',
 'lead_quality_warm']

In [35]:
leads_x_train_clean_rfe = leads_x_train_clean[rfe_col]
leads_x_train_clean_rfe.head()

,age,income,total_calls_attended,total_meetings_attended,company_estimated_revenue,lead_source_other,lead_source_referral,education_level_master,occupation_employee,occupation_unemployed,initial_response_neutral,initial_response_positive,company_size_medium,lead_quality_hot,lead_quality_warm
7072,-0.143605,-0.280340,0.135095,0.227912,-0.515709,True,False,False,True,False,False,True,True,True,False
4141,1.093861,0.902358,-0.423809,1.145328,-0.693591,False,False,False,False,False,False,True,True,True,False
6227,-0.246727,-0.411751,-0.423809,0.227912,-0.853138,True,False,False,True,False,True,False,True,False,True
9804,-0.349849,-0.849787,-1.541618,-1.606919,-1.130094,True,False,False,False,True,False,False,False,False,False
7930,-1.277948,-0.543162,-0.982714,-1.606919,-1.170302,True,False,True,False,False,False,False,False,False,False


In [36]:
leads_x_test_clean_rfe = leads_x_test_clean[rfe_col]
leads_x_test_clean_rfe.head()

,age,income,total_calls_attended,total_meetings_attended,company_estimated_revenue,lead_source_other,lead_source_referral,education_level_master,occupation_employee,occupation_unemployed,initial_response_neutral,initial_response_positive,company_size_medium,lead_quality_hot,lead_quality_warm
6139,-0.246727,-0.433653,0.135095,0.227912,-0.227495,True,False,False,True,False,False,True,True,False,True
1012,1.815716,1.822234,1.811809,0.227912,1.606008,False,False,False,False,False,False,True,True,True,False
1784,1.609471,1.077572,0.135095,0.227912,0.749087,False,False,False,False,False,False,True,True,True,False
9452,-1.174826,-0.280340,-1.541618,-1.606919,-1.173841,False,True,True,False,False,False,False,False,False,False
1329,1.815716,0.442420,-0.423809,0.227912,1.406896,False,True,False,False,False,False,True,True,False,True


In [37]:
## Train the random forest model
num_estimators = 100
min_samples = 4

rf = RandomForestClassifier(n_estimators=num_estimators,
                            min_samples_split=min_samples)
rf.fit(leads_x_train_clean, leads_y_train.values.ravel())

RandomForestClassifier(min_samples_split=4)

In [38]:
leads_y_test_predicted = rf.predict(leads_x_test_clean)
leads_y_test_predicted = pd.DataFrame(leads_y_test_predicted)

accuracy = metrics.accuracy_score(leads_y_test, leads_y_test_predicted)
#auc_score = metrics.roc_auc_score(leads_y_test, leads_y_test_predicted, multi_class='ovr')

print(accuracy)
#print(auc_score)

0.050666666666666665


In [39]:
leads_y_test_predicted = pd.DataFrame(leads_y_test_predicted)
leads_y_test_predicted.head()

,0
0,60
1,96
2,94
3,19
4,79


In [40]:
leads_y_test.head()

,lead_score
6139,64
1012,89
1784,77
9452,13
1329,85


In [41]:
lr = sm.GLM(leads_y_train,leads_x_train_clean_rfe, family = sm.families.Binomial()).fit()

ValueError: Pandas data cast to numpy dtype of object. Check apiInput data with np.asarray(data). The types seen werelead_score    int64
dtype: object and age                          float64
income                       float64
total_calls_attended         float64
total_meetings_attended      float64
company_estimated_revenue    float64
lead_source_other               bool
lead_source_referral            bool
education_level_master          bool
occupation_employee             bool
occupation_unemployed           bool
initial_response_neutral        bool
initial_response_positive       bool
company_size_medium             bool
lead_quality_hot                bool
lead_quality_warm               bool
dtype: object. The data was
      lead_score
7072          52
4141          93
6227          74
9804          42
7930          18
...          ...
3153          87
5491          65
5748          55
5271          63
6497          74

[7000 rows x 1 columns]
and
            age    income  to

In [42]:
leads_y_test_predicted = rf.predict(leads_x_test_clean)
leads_y_test_predicted = pd.DataFrame(leads_y_test_predicted)

accuracy = metrics.accuracy_score(leads_y_test, leads_y_test_predicted)
#auc_score = metrics.roc_auc_score(leads_y_test, leads_y_test_predicted, multi_class='ovr')

print(accuracy)

0.050666666666666665


In [43]:
from sklearn.metrics import accuracy_score, confusion_matrix

LR_clf = LogisticRegression(max_iter=1000)
LR_clf.fit(leads_x_train_clean_rfe, leads_y_train)
LR_pred = LR_clf.predict(leads_x_test_clean_rfe)

LR_acc = accuracy_score(leads_y_test, LR_pred)
LR_cnf = confusion_matrix(leads_y_test, LR_pred)
print('Accuracy:', LR_acc)
print('Confusion Matrix:')
print(LR_cnf)

C:\Users\hp\PycharmProjects\leadScoringModel\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.052
Confusion Matrix:
[[0 5 3 ... 0 0 0]
 [2 4 0 ... 0 0 0]
 [1 8 1 ... 0 0 0]
 ...
 [0 0 0 ... 3 8 0]
 [0 0 0 ... 3 8 0]
 [0 0 0 ... 5 7 0]]


In [44]:
leads_y_test.head()

,lead_score
6139,64
1012,89
1784,77
9452,13
1329,85


In [48]:
LR_pred[2]

87